In [163]:
"""
食べログ
"""
%xmode Verbose

import re
import sys

class Tabelog:
    """
    食べログスクレイピングクラス。場所とキーワードと条件からランキング順に10件表示。
    ランチ検索などできるように子クラスを作成し、インターフェースと実行クラスを作成予定だが、一旦この状態。
    """
    def __init__(self, place, keyword, max_price = 5000, max_check_page = 5):
        self._count = 1 #表示する店の数
        self._page = 2 #検索するページ数
        self._max_contents = 10 #検索する店の最大数
        self.param ={"sa": place, "sk": keyword} #検索する地名とキーワード
        self.max_price = max_price #表示するお店の最大金額。デフォルト5000円
        self.max_check_page = max_check_page #検索する最大ページ数。デフォルト5ページ
        
        
        #検索する地名とキーワードでランキング順に並べて初期化
        soup = self.setup(self.param)
        #1ページ目で条件に合うものを表示
        self._count = self.select_store(soup, self._count, self.max_price)
        #検索する店の最大数に行くまで検索
        while self._count < self._max_contents:
            #print("self._count:"+str(self._count))
            self.re_select(soup, self._page, self.max_check_page)


    def setup(self, param):
        html_doc = requests.get('https://tabelog.com/rst/rstsearch/', params = param).content
        #print(html_doc.url)
        soup = BeautifulSoup(html_doc, 'html.parser') # BeautifulSoupの初期化
        sorted_pages = soup.find("a",{"class": "navi-rstlst__label navi-rstlst__label--rank"})
        html = requests.get(sorted_pages["href"]).content
        #print(requests.get(sorted_pages["href"]).url)
        soup = BeautifulSoup(html, 'html.parser') # BeautifulSoupの初期化
        return soup
    
    def re_select(self, soup, page, max_check_page):
        check_pages = soup.find_all("a",{"class": "c-pagination__num"}) 
        for check_page in check_pages:
            if check_page.text == str(page) and page <= max_check_page:
                html = requests.get(check_page["href"]).content
                soup = BeautifulSoup(html, 'html.parser')
                #print(check["href"])
                self._count = self.select_store(soup,self._count,self.max_price)
                self._page += 1
            #else:
                #sys.exit(0)
    

    def select_store(self, soup, count, max_price):
        tags =soup.find_all("div",{"class": "list-rst__wrap js-open-new-window"}) 
        #print(tags)
        for tag in tags:
            if count <= 10:
                flag= True #最大予算フラグ
                budgets = tag.find_all("li",{"class": "c-rating c-rating--s list-rst__budget-item"})
                for budget in budgets:
                    budget_text = budget.text.split()[0]
                    pat = r'[0-9]+' #抽出するパターン：数字
                    num = budget_text.replace(",","")
                    price_list = re.findall(pat, num)  #数字を抽出してリスト化
                    for price in price_list:
                        if int(price) >= max_price:
                            flag = False
                            break

                if flag == True:
                    print(str(count) +":"+tag.a.text)
                    print(tag.a["href"])
                    print("場所:" + tag.find("span",{"class": "list-rst__area-genre cpy-area-genre"}).text)
                    try:
                        print("評価:" + tag.find("span",{"class": "c-rating__val c-rating__val--strong list-rst__rating-val"}).text)
                    except AttributeError:
                        pass
                    for budget in budgets:
                        print(budget.text.split()[0])
                    try:
                        print("定休日:" + tag.find("span",{"class": "list-rst__holiday-datatxt cpy-holiday-datatxt"}).text)
                    except AttributeError:
                        pass
                    print()
                    count += 1
        return count

#この文言を最終的にLINE_BOTからとる。
test = Tabelog("渋谷","寿司")

Exception reporting mode: Verbose
1:KINKA sushi bar izakaya 渋谷
https://tabelog.com/tokyo/A1303/A130301/13199924/
場所: 渋谷駅 530m / 居酒屋、寿司、日本酒バー
評価:3.51
夜の予算:￥4,000～￥4,999
昼の予算:～￥999

2:梅丘寿司の美登利総本店 渋谷店
https://tabelog.com/tokyo/A1303/A130301/13004624/
場所: 渋谷駅 318m / 寿司
評価:3.49
夜の予算:￥4,000～￥4,999
昼の予算:￥2,000～￥2,999
定休日:1月1日

3:回し寿司 活 西武渋谷店
https://tabelog.com/tokyo/A1303/A130301/13135066/
場所: 渋谷駅 241m / 回転寿司
評価:3.49
夜の予算:￥2,000～￥2,999
昼の予算:￥1,000～￥1,999
定休日:年中無休※西武と順ずる

4:居酒屋 肴とり
https://tabelog.com/tokyo/A1303/A130301/13198186/
場所: 神泉駅 303m / 居酒屋、魚介料理・海鮮料理、寿司
評価:3.44
夜の予算:￥3,000～￥3,999
昼の予算:～￥999

5:すし台所家 渋谷本店
https://tabelog.com/tokyo/A1303/A130301/13037261/
場所: 神泉駅 264m / 回転寿司、寿司、魚介料理・海鮮料理
評価:3.43
夜の予算:￥1,000～￥1,999
昼の予算:～￥999
定休日:年中無休

6:グルメ廻転寿司 　まぐろ問屋　三浦三崎港 恵み 渋谷ヒカリエ店
https://tabelog.com/tokyo/A1303/A130301/13140077/
場所: 渋谷駅 188m / 回転寿司
評価:3.38
夜の予算:￥3,000～￥3,999
昼の予算:￥1,000～￥1,999
定休日:休館日以外は原則営業。

7:笹八 渋谷店
https://tabelog.com/tokyo/A1303/A130301/13007483/
場所: 渋谷駅 61m / おにぎり、寿司
評価:3.38